# Mission To Mars - Homework
## by Michael Dowlin
### 1/12/2020

## Import Dependencies

In [1]:
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests
from splinter import Browser

## Scraping
Scrape the Mars website and setup the bs, lol

In [2]:
# Defining the url for the mars website
url = 'https://mars.nasa.gov/news/'

# return the rendered html
response = requests.get(url)

# create the bs object
space_bs = bs(response.text, 'html.parser')

### Nasa Mars News
#### Using bs to look at article divs
To get the first article's title, I used the in-browser tool to find the class "content_title", which is used by the div containing the news article title.  

In [3]:
# return the first article div
title_div = space_bs.find('div', class_='content_title')

# check out the results...
title_div

<div class="content_title">
<a href="/news/8508/nasa-invites-students-to-name-mars-2020-rover/">
NASA Invites Students to Name Mars 2020 Rover
</a>
</div>

#### Get the article title

In [4]:
# grab the text from the only anchor (a) tag in the div, stripping the new line
article_title = title_div.a.text.strip('\n')

# printing out the top article title
print(f'The first new\'s article title is: "{article_title}"')

The first new's article title is: "NASA Invites Students to Name Mars 2020 Rover"


#### Now get the paragraph description, and paragraph text

In [5]:
# div containing the paragraph, found by looking for the class "rollover_description_inner"
p_div = space_bs.find('div',class_='rollover_description_inner')

# grab the text from the div, stripping out the new line
article_p = p_div.text.strip('\n')

# printing the paragraph for the article
print(f'The paragraph text describing the article is: \n"{article_p}"')

The paragraph text describing the article is: 
"Through Nov. 1, K-12 students in the U.S. are encouraged to enter an essay contest to name NASA's next Mars rover."


### JPL Mars Space Images - Featured Image
Using a chrome-driver for the splinter portion, and bs to get the image and stuff

In [9]:
# Using splinter, setup a Chrome browser
browser = Browser('chrome')

# navigate to image page
browser.visit('https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars')

# # parse the html with some bs
image_bs = bs(browser.html, 'html.parser')
# image_bs

#### Getting the image url for the variable "featured_image_url"

In [21]:
# grab the image div
img_div = image_bs.find('div', class_="img")

# find the img object
img = img_div.find('img')

# get the image object's source
img_src = img['src']

# checking out the string, turns out a partial URL path
# print(img_src)

# since it is not a full URL path, add the base (jpl.nasa.gov) to the source
# setting up the image base url
base_url = 'https://www.jpl.nasa.gov'

# setting the featured image url (base + image_id)
featured_image_url = base_url + img_src

# grabbing the image title too, it may come in handy later
featured_image_title = img['title']

# checking work
print(f'Featured image\nTitle: {featured_image_title}\nURL: {featured_image_url}')

Featured image
Title: Sacra Mensa
URL: https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA23634-640x350.jpg


### Mars Facts


In [31]:
# Using splinter, setup a Chrome browser
browser = Browser('chrome')

# navigate to image page
browser.visit('https://space-facts.com/mars/')

# # parse the html with some bs
facts_bs = bs(browser.html, 'html.parser')

#### Just the facts

In [61]:
# return the div that contains the facts
facts_div = facts_bs.find('div', id='facts')

# grab the factrs unordered list
facts_ul = facts_div.find('ul')

# create a list of list items in the ul
facts_li = facts_ul.find_all('li')

# list to store the facts
mars_facts = []
# dummy id
fact_id = 0

# loop through the facts, creating a dictionairy for each row, and appending to the facts list
for fact_li in facts_li:
    # the fact title is in the stron tag, so grab that text
    fact_title = fact_li.find('strong').text
    
    # make a dictionary of the object
    fact = {
        'fact_id': fact_id,
        'fact_title': fact_title,
        'fact': fact_li.text.strip('\n')
    }
    
    # add to list
    mars_facts.append(fact)
    
    # increment the id counter
    fact_id +=1
    
# convert list of dictionairies to a dataframe
facts_df = pd.DataFrame(mars_facts)

# checking it out
# facts_df.head()

# create the html string for the table of facts, adding the bootstrap table classes
facts_table = facts_df.to_html(classes='table table-striped')

# testing
facts_table


'<table border="1" class="dataframe table table-striped">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>fact_id</th>\n      <th>fact_title</th>\n      <th>fact</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>0</td>\n      <td>Mars and Earth have approximately the same landmass.</td>\n      <td>Mars and Earth have approximately the same landmass.Even though Mars has only 15% of the Earth’s volume and just over 10% of the Earth’s mass, around two thirds of the Earth’s surface is covered in water. Martian surface gravity is only 37% of the Earth’s (meaning you could leap nearly three times higher on Mars).</td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>1</td>\n      <td>Mars is home to the tallest mountain in the solar system.</td>\n      <td>Mars is home to the tallest mountain in the solar system.Olympus Mons, a shield volcano, is 21km high and 600km in diameter. Despite having formed over billions of years, evidence fr

### Mars Hemispheres

In [84]:
# Using splinter, setup a Chrome browser
browser = Browser('chrome')

# navigate to image page
browser.visit('https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars')

# parse the html with some bs
hemi_bs = bs(browser.html, 'html.parser')

In [94]:
# find the four anchor tags for each hemisphere
hemi_as = hemi_bs.find_all('a', class_='itemLink')

# counter for id
hemi_id = 0

# base url for hemisphere page
base_url = 'https://astrogeology.usgs.gov'

# list to store dicitonaries (rows)
hemispheres = []

len(hemi_as)

# there are two links per hemisphere, i only want to grab one
hemi_link_cntr = 0

# list to store the hemisphere stuff
hemispheres = []

# loop through the 4 hemis
for hemi_a in hemi_as:
    
    if hemi_link_cntr == 1:
        
        # grab this hemisphere's href (link)
        img_url = hemi_a['href']
    
        # go to the hemi page
        hemi_url = base_url + img_url
    
        # going to page
        browser.visit(hemi_url)
        
        # bs on the hemisphere page
        single_hemi_bs = bs(browser.html, 'html.parser')
        
        # find the a-tags, loop through until the one with 'original' is found
        a_tags = single_hemi_bs.find_all('a','')
        for a_tag in a_tags:
            
            a_text = a_tag.text
            
            if a_text == 'Original':
                
                img_url = a_tag['href']
                
       
        # get the h2 tag with the class of "title"
        title_tag = single_hemi_bs.find('h2', class_='title')

        # get the text
        title = title_tag.text
        
        # create the dictionary
        hemisphere = {
            "hemi_id": hemi_id,
            "title": title,
            "img_url": img_url
        }
        
        hemispheres.append(hemisphere)
        
        # increment the counter
        hemi_id += 1
        
        # reset the counter
        hemi_link_cntr = 0
    
    else:
        
        # increment the counter
        hemi_link_cntr = 1
            
print(hemispheres)


[{'hemi_id': 0, 'title': 'Cerberus Hemisphere Enhanced', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif'}, {'hemi_id': 1, 'title': 'Schiaparelli Hemisphere Enhanced', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif'}, {'hemi_id': 2, 'title': 'Syrtis Major Hemisphere Enhanced', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif'}, {'hemi_id': 3, 'title': 'Valles Marineris Hemisphere Enhanced', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif'}]


In [95]:
# converting it to a dataframe just to check it out
hemispheres_df = pd.DataFrame(hemispheres)

hemispheres_df.head()

,hemi_id,title,img_url
0,0,Cerberus Hemisphere Enhanced,http://astropedia.astrogeology.usgs.gov/downlo...
1,1,Schiaparelli Hemisphere Enhanced,http://astropedia.astrogeology.usgs.gov/downlo...
2,2,Syrtis Major Hemisphere Enhanced,http://astropedia.astrogeology.usgs.gov/downlo...
3,3,Valles Marineris Hemisphere Enhanced,http://astropedia.astrogeology.usgs.gov/downlo...


In [96]:
mars_updates = {
    'latest_news_title': article_title,
    'latest_news_desc': article_p,
    'featured_image_url': featured_image_url,
    'featured_image_title': featured_image_title,
    'facts_table': facts_table,
    'hemispheres' : hemispheres
}

print(mars_updates)

{'latest_news_title': 'NASA Invites Students to Name Mars 2020 Rover', 'latest_news_desc': "Through Nov. 1, K-12 students in the U.S. are encouraged to enter an essay contest to name NASA's next Mars rover.", 'featured_image_url': 'https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA23634-640x350.jpg', 'featured_image_title': 'Sacra Mensa', 'facts_table': '<table border="1" class="dataframe table table-striped">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>fact_id</th>\n      <th>fact_title</th>\n      <th>fact</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>0</td>\n      <td>Mars and Earth have approximately the same landmass.</td>\n      <td>Mars and Earth have approximately the same landmass.Even though Mars has only 15% of the Earth’s volume and just over 10% of the Earth’s mass, around two thirds of the Earth’s surface is covered in water. Martian surface gravity is only 37% of the Earth’s (meaning you could leap ne